In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2 as cv
import glob
import yaml
import cv2
import time
import matplotlib
%matplotlib inline
matplotlib.rc('image', cmap = 'gray')

# Pupil Eye Detection

In [4]:
# Initialize the webcam
cap = cv2.VideoCapture(0)

# Load the pre-trained face and eye detection models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

prev_time = time.time()

while True:
    ret, frame = cap.read()
    
    if not ret or frame is None:  # Ensure frame is valid
        print("Error: Could not capture frame.")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Detect eyes within the face region
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))
        
        for (ex, ey, ew, eh) in eyes[:2]:  # Process only the first two detected eyes
            eye_gray = roi_gray[ey:ey+eh, ex:ex+ew]
            eye_color = roi_color[ey:ey+eh, ex:ex+ew]

            # Apply thresholding to isolate the pupil
            _, thresh = cv2.threshold(eye_gray, 50, 255, cv2.THRESH_BINARY_INV)

            # Find contours and get the largest one, assuming it's the pupil
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                (cx, cy), radius = cv2.minEnclosingCircle(largest_contour)
                if radius < ew // 4:  # Ensure the detected circle is within the eye region
                    #cv2.circle(eye_color, (int(cx), int(cy)), int(radius), (255, 0, 0), 2)
                    cv2.circle(eye_color, (int(cx), int(cy)), 2, (0, 255, 0), -1)  # Pupil center
    
    # Calculate frame rate
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    prev_time = curr_time
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow('Pupil Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()